In [19]:
import time
import os
import glob # To easily find files matching a pattern
# --- Helper Functions ---

def parse_set_cover_instance(filename):
    """
    Parses the input file for the Set Cover instance.
    Args:
        filename (str): The path to the input .in file.
    Returns:
        tuple: (universe_size, subsets)
               - universe_size (int): Number of elements in the universe (n).
               - subsets (list): A list of sets, where each set contains the elements
                                 it covers. Indices correspond to the subset index + 1.
    """
    subsets = []
    universe_size = 0
    num_subsets = 0
    try:
        with open(filename, 'r') as f:
            # Read the first line: n m
            line1 = f.readline().strip().split()
            if len(line1) == 2:
                universe_size = int(line1[0])
                num_subsets = int(line1[1]) #
            else:
                 print(f"Error: First line format incorrect in {filename}")
                 return None, None

            # Read subsequent lines for subsets
            for i in range(num_subsets): # Use index i for subset number
                 line = f.readline().strip().split()
                 # First element is size |Si|, rest are elements
                 # Convert elements to integers, handle potential errors
                 try:
                      if not line: # Handle empty lines if they occur
                          print(f"Warning: Empty line {i+2} encountered in {filename}")
                          continue
                      # Expecting size followed by elements
                      # size = int(line[0]) # We don't strictly need the size, just the elements
                      elements = set(map(int, line[1:]))
                      subsets.append(elements)
                 except (ValueError, IndexError) as e:
                      print(f"Error parsing subset line {i+2} in {filename}: {e}. Line content: '{' '.join(line)}'")
                      return None, None # Abort parsing on error


        # Basic validation
        if len(subsets) != num_subsets:
             # This might trigger if empty lines were skipped, adjust logic if needed
             print(f"Warning: Mismatch between expected ({num_subsets}) and read ({len(subsets)}) subsets in {filename}")
             # Decide if this is critical; for now, allow continuing

        return universe_size, subsets
    except FileNotFoundError:
        print(f"Error: Input file not found: {filename}")
        return None, None
    except Exception as e:
        print(f"Error parsing file {filename}: {e}")
        return None, None


def read_optimal_value(filename):
    """
    Reads the optimal solution value from the .out file.
    MODIFIED: Assumes the first line contains space-separated indices of the
    optimal solution, and the optimal VALUE is the COUNT of these indices.
    Args:
        filename (str): The path to the output .out file.
    Returns:
        int or None: The optimal number of subsets (count of indices on line 1),
                     or None if file not found/readable or format is unexpected.
    """
    try:
        with open(filename, 'r') as f:
            first_line = f.readline().strip()
            # Split the first line by space - these are assumed to be indices
            parts = first_line.split()
            # The optimal value is the number of indices listed
            # Handle empty line case: if parts is empty, count is 0.
            opt_value = len(parts)
            return opt_value
    except FileNotFoundError:
        print(f"Warning: Optimal value file not found: {filename}. Cannot calculate RelErr.")
        return None
    except Exception as e:
        # General catch-all for other file reading errors
        print(f"Error reading optimal value file {filename}: {e}")
        return None


def write_solution_file(filename_base, method, cutoff, seed, solution_indices, quality):
    """
    Writes the solution file in the specified format.
    Args:
        filename_base (str): Base name of the instance (e.g., 'test1').
        method (str): Algorithm used ('Approx' for greedy).
        cutoff (int): Cutoff time (relevant for other algos, included for consistency).
        seed (int): Random seed (relevant for other algos).
        solution_indices (list): List of 1-based indices of selected subsets.
        quality (int): The number of subsets selected (size of the cover).
    """
    # Format: <instance>_<method>_<cutoff>[-<randSeed>].sol
    # Seed is omitted for deterministic methods like greedy
    sol_filename = f"{filename_base}_{method}_{cutoff}.sol"

    try:
        with open(sol_filename, 'w') as f:
            # Line 1: quality (minimal size)
            f.write(f"{quality}\n")
            # Line 2: list of indices (space-separated) - Ensure indices are strings
            f.write(" ".join(map(str, solution_indices)) + "\n")
        # print(f"Solution file written: {sol_filename}") # Optional: uncomment for verbose output
    except Exception as e:
        print(f"Error writing solution file {sol_filename}: {e}")


# --- Greedy Set Cover Algorithm ---

def greedy_set_cover(universe_size, subsets):
    """
    Implements the Greedy Set Cover algorithm.
    Args:
        universe_size (int): Number of elements in the universe (n).
        subsets (list): A list of sets, where each set contains the elements it covers.
                        Indices in this list are 0-based.
    Returns:
        tuple: (cover_indices, covered_elements_count, execution_time)
               - cover_indices (list): 1-based indices of the subsets in the final cover.
               - covered_elements_count (int): Number of unique elements covered.
               - execution_time (float): Time taken in seconds.
    """
    start_time = time.time()

    # Elements that still need to be covered (1 to universe_size)
    uncovered_elements = set(range(1, universe_size + 1))
    # The collection of subsets chosen for the cover (store their 1-based indices)
    cover_indices = []
    # Keep track of subsets still available for selection.
    # Store as list of tuples: (original_1_based_index, set_of_elements)
    # We need the original index (1-based) for the final output.
    remaining_subsets = [(i + 1, s.copy()) for i, s in enumerate(subsets)]
    # Keep a set of selected 1-based indices to prevent duplicates if logic allows
    selected_indices_set = set()

    while uncovered_elements and remaining_subsets:
        best_subset_original_index = -1
        elements_covered_by_best = set()
        max_newly_covered_count = -1
        best_subset_internal_index = -1 # Index within remaining_subsets list

        # Find the subset that covers the most *currently uncovered* elements
        for i, (original_index, subset_elements) in enumerate(remaining_subsets):
            newly_covered = subset_elements.intersection(uncovered_elements)
            num_newly_covered = len(newly_covered)

            # Greedy choice: Maximize newly covered elements.
            # Tie-breaking (optional but good practice): smallest original index, or smallest set size?
            # Standard greedy usually doesn't specify tie-breaking, pick first max found.
            if num_newly_covered > max_newly_covered_count:
                max_newly_covered_count = num_newly_covered
                elements_covered_by_best = newly_covered
                best_subset_original_index = original_index
                best_subset_internal_index = i # Store index *in remaining_subsets*

        # Check if we could find any subset covering new elements
        if max_newly_covered_count <= 0:
            # No remaining subset can cover any more uncovered elements.
            if uncovered_elements:
                 # This is normal if a full cover is impossible with the given subsets
                 pass # Don't print warning unless it's unexpected
            break # Stop the algorithm

        # Add the best subset found to our cover (use original 1-based index)
        if best_subset_original_index not in selected_indices_set:
             cover_indices.append(best_subset_original_index)
             selected_indices_set.add(best_subset_original_index)

        # Remove the covered elements from the uncovered set
        uncovered_elements.difference_update(elements_covered_by_best)

        # Remove the chosen subset from *further* consideration in this run.
        # Pop using the index within the remaining_subsets list.
        if best_subset_internal_index != -1:
            remaining_subsets.pop(best_subset_internal_index)
        else:
             # This case should ideally not be reached if max_newly_covered_count > 0
             print(f"Error logic: Could not find internal index for chosen subset {best_subset_original_index}")
             break


    end_time = time.time()
    execution_time = end_time - start_time

    # Final check on coverage (informational)
    final_covered_count = universe_size - len(uncovered_elements)
    if uncovered_elements:
         print(f"Info: Algorithm finished. {len(uncovered_elements)} elements remain uncovered. (Universe size: {universe_size}, Covered: {final_covered_count})")

    # Return 1-based indices, count of elements actually covered, and time
    return sorted(cover_indices), final_covered_count, execution_time # Sort indices for consistent output


# --- Main Execution Logic ---

def run_experiment(data_dir, instance_pattern, method, cutoff, seed=None):
    """
    Runs the greedy algorithm on all instances matching the pattern.
    Args:
        data_dir (str): Directory containing the data files.
        instance_pattern (str): Glob pattern for instance files (e.g., 'test*.in').
        method (str): Algorithm identifier ('Approx').
        cutoff (int): Cutoff time (nominal for greedy).
        seed (int): Random seed (nominal for greedy).
    """
    results = []
    # Ensure output directory exists
    output_dir = "output"
    if not os.path.exists(output_dir):
         os.makedirs(output_dir)

    input_files = sorted(glob.glob(os.path.join(data_dir, instance_pattern)))

    if not input_files:
        print(f"Warning: No input files found for pattern: {instance_pattern} in {data_dir}")
        return results

    print(f"\n--- Running {method} on {instance_pattern} ---")

    for in_file in input_files:
        base_name = os.path.basename(in_file).replace('.in', '')
        out_file = os.path.join(data_dir, base_name + '.out')
        print(f"Processing: {base_name}")

        # 1. Parse instance
        universe_size, subsets = parse_set_cover_instance(in_file)
        if universe_size is None or subsets is None:
            print(f"Skipping {base_name} due to parsing error.")
            results.append({'Instance': base_name, 'Time': 0, 'Quality': 'Error', 'Optimal': 'N/A', 'RelErr': 'N/A', 'Method': method})
            continue

        # 2. Read optimal value (for RelErr calculation) - uses updated logic
        opt_value = read_optimal_value(out_file)

        # 3. Run Greedy Algorithm
        cover_indices, covered_count, exec_time = greedy_set_cover(universe_size, subsets)
        quality = len(cover_indices) # The size of the cover found by the algorithm

        # Check if algorithm covered everything (important for correctness)
        if covered_count < universe_size:
            print(f"  WARNING: Greedy algorithm did not cover all elements for {base_name}. Universe: {universe_size}, Covered: {covered_count}")


        # 4. Calculate Relative Error
        rel_err = None
        opt_value_display = 'N/A' # For printing
        if opt_value is not None:
            opt_value_display = opt_value
            if opt_value > 0:
                 rel_err = (quality - opt_value) / float(opt_value) # Ensure float division
            elif opt_value == 0: # Handle optimal = 0 case
                 # If optimal is 0, RelErr is 0 if quality is also 0, else Inf.
                 rel_err = 0.0 if quality == 0 else float('inf')
            else: # opt_value < 0 ? Should not happen.
                 print(f"  Warning: Optimal value is negative ({opt_value}) for {base_name}. RelErr calculation skipped.")
                 rel_err = None # Mark as not calculable

            print(f"  Result Quality: {quality}, Optimal: {opt_value_display}, RelErr: {rel_err if rel_err is not None else 'N/A':.4f}, Time: {exec_time:.4f}s")

        else: # opt_value is None (error reading file)
             print(f"  Result Quality: {quality}, Optimal: N/A, RelErr: N/A, Time: {exec_time:.4f}s")


        # 5. Write solution file
        sol_file_base = os.path.join(output_dir, base_name) # Path for output file
        write_solution_file(sol_file_base, method, cutoff, seed, cover_indices, quality)

        # Store results for potential later analysis/table generation
        results.append({
            'Instance': base_name,
            'Time': exec_time,
            'Quality': quality,
            'Optimal': opt_value, # Store the read optimal value
            'RelErr': rel_err,
            'CoveredElements': covered_count, # Store how many elements were covered
            'UniverseSize': universe_size,
            'Method': method
        })

    return results

# --- Configuration and Execution ---

DATA_DIRECTORY = "data"  # Assumes 'data' subdirectory exists
CUTOFF_TIME = 600      # Nominal cutoff time (seconds), as required by spec, though greedy is usually fast
RANDOM_SEED = None     # Not applicable for deterministic greedy
METHOD_NAME = "Approx" # Algorithm identifier for file naming

# Run experiments on all datasets
all_results = []
# Test instances
all_results.extend(run_experiment(DATA_DIRECTORY, "test*.in", METHOD_NAME, CUTOFF_TIME, RANDOM_SEED))
# Small instances
all_results.extend(run_experiment(DATA_DIRECTORY, "small*.in", METHOD_NAME, CUTOFF_TIME, RANDOM_SEED))
# Large instances
all_results.extend(run_experiment(DATA_DIRECTORY, "large*.in", METHOD_NAME, CUTOFF_TIME, RANDOM_SEED))

print("\n--- Experiment Complete ---")

# --- Optional: Display Results Table (using pandas) ---
# You might need to install pandas: !pip install pandas
try:
    import pandas as pd
    df = pd.DataFrame(all_results)

    # Format RelErr to handle potential None or inf values before displaying
    def format_rel_err(x):
        if x is None: return 'N/A'
        if x == float('inf'): return 'inf'
        if isinstance(x, (int, float)): return f"{x:.4f}"
        return str(x) # Fallback

    df['RelErr_Display'] = df['RelErr'].apply(format_rel_err)

    # Format Time
    df['Time'] = df['Time'].apply(lambda x: f"{x:.4f}" if isinstance(x, (int, float)) else str(x))

    print("\n--- Results Summary ---")
    # Display key columns including the formatted RelErr
    print(df[['Instance', 'Method', 'Time', 'Quality', 'Optimal', 'RelErr_Display']])

except ImportError:
    print("\nPandas not installed. Skipping results table display.")
    print("Raw results list:")
    # Print basic results if pandas is not available
    for res in all_results:
         rel_err_str = 'N/A'
         if res['RelErr'] is not None:
             if res['RelErr'] == float('inf'): rel_err_str = 'inf'
             elif isinstance(res['RelErr'], (int, float)): rel_err_str = f"{res['RelErr']:.4f}"
             else: rel_err_str = str(res['RelErr'])

         print(f"  {res['Instance']}: Time={res['Time']:.4f}, Quality={res['Quality']}, Optimal={res['Optimal']}, RelErr={rel_err_str}")


--- Running Approx on test*.in ---
Processing: test1
  Result Quality: 2, Optimal: 2, RelErr: 0.0000, Time: 0.0000s
Processing: test2
  Result Quality: 3, Optimal: 3, RelErr: 0.0000, Time: 0.0000s
Processing: test3
  Result Quality: 7, Optimal: 7, RelErr: 0.0000, Time: 0.0000s
Processing: test4
  Result Quality: 5, Optimal: 5, RelErr: 0.0000, Time: 0.0000s
Processing: test5
  Result Quality: 5, Optimal: 5, RelErr: 0.0000, Time: 0.0000s

--- Running Approx on small*.in ---
Processing: small1
  Result Quality: 5, Optimal: 5, RelErr: 0.0000, Time: 0.0000s
Processing: small10
  Result Quality: 3, Optimal: 3, RelErr: 0.0000, Time: 0.0000s
Processing: small11
  Result Quality: 5, Optimal: 5, RelErr: 0.0000, Time: 0.0000s
Processing: small12
  Result Quality: 4, Optimal: 4, RelErr: 0.0000, Time: 0.0000s
Processing: small13
  Result Quality: 3, Optimal: 3, RelErr: 0.0000, Time: 0.0000s
Processing: small14
  Result Quality: 3, Optimal: 3, RelErr: 0.0000, Time: 0.0000s
Processing: small15
  Res